# 01 — pytest Fundamentals

This notebook covers:
- Running pytest from a notebook
- Writing test functions and classes
- Rich assertions, `pytest.raises`
- Markers: `skip`, `xfail`, `parametrize`
- Test discovery rules

## Setup

In [ ]:
import subprocess, sys, os, textwrap, tempfile, pathlib

# Helper: write a temp test file and run pytest on it
def run_pytest(test_code: str, extra_args: list[str] | None = None):
    """Write test_code to a temp file and run pytest. Returns (stdout, returncode)."""
    with tempfile.TemporaryDirectory() as td:
        p = pathlib.Path(td) / "test_tmp.py"
        p.write_text(textwrap.dedent(test_code))
        cmd = [sys.executable, "-m", "pytest", str(p), "-v", "--tb=short", "--no-header"]
        if extra_args:
            cmd.extend(extra_args)
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=30)
        output = result.stdout + result.stderr
        print(output)
        return result.returncode

print("Helper ready.")

## 1. Your First Test

In [ ]:
rc = run_pytest('''
def test_addition():
    assert 1 + 1 == 2

def test_string():
    assert "hello".upper() == "HELLO"
''')
assert rc == 0, "Tests should pass"
print("\nAll passed!")

## 2. Rich Assertion Messages

pytest rewrites `assert` to show detailed diffs — no need for `assertEqual`.

In [ ]:
# This test FAILS on purpose to show the diff output
rc = run_pytest('''
def test_list_diff():
    result = [1, 2, 3, 4]
    expected = [1, 2, 99, 4]
    assert result == expected
''')
assert rc != 0, "Should fail"
print("\n^ Notice how pytest shows the exact index that differs.")

## 3. Testing Exceptions with `pytest.raises`

In [ ]:
rc = run_pytest('''
import pytest

def divide(a, b):
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b

def test_divide_ok():
    assert divide(10, 2) == 5.0

def test_divide_error():
    with pytest.raises(ValueError, match="Cannot divide by zero"):
        divide(1, 0)

def test_type_error():
    with pytest.raises(TypeError):
        divide("a", "b")
''')
assert rc == 0
print("\nException tests passed!")

## 4. Markers

Markers let you tag tests and run subsets.

In [ ]:
rc = run_pytest('''
import pytest

@pytest.mark.skip(reason="not implemented yet")
def test_future():
    assert False  # never runs

@pytest.mark.xfail(reason="known bug")
def test_known_bug():
    assert 1 == 2  # expected to fail

def test_normal():
    assert True
''')
assert rc == 0
print("\nskip -> skipped, xfail -> xfail (expected failure), normal -> passed")

## 5. `parametrize` — Multiple Inputs, One Test

In [ ]:
rc = run_pytest('''
import pytest

@pytest.mark.parametrize("x, y, expected", [
    (1, 1, 2),
    (0, 0, 0),
    (-1, 1, 0),
    (100, 200, 300),
])
def test_add(x, y, expected):
    assert x + y == expected
''')
assert rc == 0
print("\n4 parameterized cases, all passed!")

## 6. Test Classes

Group related tests. No `__init__` needed.

In [ ]:
rc = run_pytest('''
class TestStringMethods:
    def test_upper(self):
        assert "hello".upper() == "HELLO"

    def test_split(self):
        assert "a,b,c".split(",") == ["a", "b", "c"]

    def test_strip(self):
        assert "  hi  ".strip() == "hi"
''')
assert rc == 0
print("\nTest class with 3 methods — all passed!")

## 7. Test Discovery Rules

pytest finds tests automatically by convention:
- Files: `test_*.py` or `*_test.py`
- Functions: `test_*`
- Classes: `Test*` (no `__init__`)

Files NOT matched: `helper.py`, `utils.py`, `conftest.py` (special purpose).

## Key Takeaways

1. Plain `assert` gives rich diffs — no `assertEqual` needed
2. `pytest.raises(ExcType, match=...)` for exception testing
3. `@pytest.mark.parametrize` eliminates copy-paste tests
4. Markers (`skip`, `xfail`, custom) control which tests run
5. Test discovery is convention-based — follow naming rules